### 3.1 Bombcell

In [ ]:
# Replace with your kilosort directory
well_id = 'well010'
ks_dir = 'D:/Master Thesis/Data/EI_iNeurons/241218/T002523/Network/well000/sorter_output/' #Path(save_root) / well_id / 'sorter_output'

# Set bombcell's output directory
save_path = Path(ks_dir) / "bc_output_test"
if not save_path.exists():
    save_path.mkdir(parents=True, exist_ok=True)

print(f"Using kilosort directory: {ks_dir}")

param = bc.get_default_parameters(ks_dir, kilosort_version=2)

# Modify some parameters
param['ephys_sample_rate'] = 10000  # Sample rate in Hz

h5 = h5py.File(rec_path)
rec_name = list(h5['wells'][well_id].keys())[0]
rec = si.MaxwellRecordingExtractor(rec_path, stream_id=well_id, rec_name=rec_name)

param['nChannels'] = rec.get_num_channels()
param['raw_data_file'] = Path(ks_dir) / "recording.dat"
param['ephysKilosortPath']

pprint(param)

(quality_metrics, param, unit_type, unit_type_string) = bc.run_bombcell(ks_dir, save_path, param)

In [ ]:
quality_metrics_table = pd.DataFrame(quality_metrics)
quality_metrics_table.insert(0, 'Bombcell_unit_type', unit_type_string)
quality_metrics_table

In [ ]:
gui = bc.unit_quality_gui(
    ks_dir=ks_dir,
    quality_metrics=quality_metrics,
    unit_types=unit_type,
    param=param,
    save_path=save_path,
)

#### Bombcell Philipp

In [ ]:
from mxw_bombcell.generate_sorting_path_list import generate_sorting_path_list
from mxw_bombcell.infer_sampling_rate import infer_sampling_rate
from mxw_bombcell.bombcell_to_phy import bombcell_to_phy


# Suppress all warnings
warnings.filterwarnings('ignore')

qc_root_path = 'D:/Master Thesis/Data/EI_iNeurons'
path_logic = ['24*', 'T002523', 'Network', 'well*', 'sorter_output']

sorting_path_list = generate_sorting_path_list(qc_root_path, path_logic)
print(f"Found {len(sorting_path_list)} sortings")

# PARAMS
rerun = 1  # 1 to re-run, 0 to skip if metrics are present
gain_to_uV = 6.2
verbose = 0

for i, sorting_path in enumerate(sorting_path_list):
    ephysKilosortPath = sorting_path
    sampling_rate = infer_sampling_rate(ephysKilosortPath)
    
    kilosortVersion = 2 # Change if using Kilosort4
    savePath = os.path.join(ephysKilosortPath, "bc_output")

    param = bc.get_default_parameters(kilosort_path=ephysKilosortPath, kilosort_version=kilosortVersion, gain_to_uV=gain_to_uV)

    param['extractRaw'] = 0
    param['ephys_sample_rate'] = sampling_rate

    param['nSyncChannels'] = 0
    param['removeDuplicateSpikes'] = 0
    param['minWvDuration'] = 200
    param['maxWvDuration'] = 1500
    param['tauR_valuesMin'] = 0.001
    param['tauR_valuesMax'] = 0.003
    param['maxRPVviolations'] = 0.05
    param['verbose'] = verbose
    param['computeDistanceMetrics'] = 1

    qMetricsExist = (len(glob(os.path.join(savePath, 'qMetric*.mat'))) > 0) or \
                    (os.path.exists(os.path.join(savePath, 'templates._bc_qMetrics.parquet')))

    if not qMetricsExist or rerun:
        spikeTimes_samples, spikeTemplates, templateWaveforms, templateAmplitudes, pcFeatures, \
        pcFeatureIdx, channelPositions = bc.load_ephys_data(ephysKilosortPath)
        
        if len(spikeTimes_samples) > 0 and param['ephys_sample_rate'] > 0:
            param['minNumSpikes'] = round(np.max(spikeTimes_samples) / param['ephys_sample_rate'] / 20)
        else:
            param['minNumSpikes'] = 0

        param['nChannels'] = channelPositions.shape[0] if channelPositions is not None else 0





        (
            qMetrics,
            param,
            unitType,
            unitTypeString,
        ) = bc.run_bombcell(
            ephysKilosortPath, savePath, param
        )


        """
        qMetric, runtimes = bc.get_all_quality_metrics(param=param, 
                                                       unique_templates=None,
                                                       spike_times_seconds=spikeTimes_samples, 
                                                       spike_clusters=spikeTemplates, 
                                                       template_waveforms=templateWaveforms, 
                                                       template_amplitudes=templateAmplitudes,
                                                       time_chunks=None,
                                                       pc_features=pcFeatures, 
                                                       pc_features_idx=pcFeatureIdx,
                                                       channel_positions=channelPositions, 
                                                       save_path=savePath)
        
        unitType, unitTypeString = bc.get_quality_unit_type(param, qMetric)
        """


bc_path_list = [os.path.join(sp, "bc_output") for sp in sorting_path_list]

overwrite = True
bombcell_to_phy(bc_path_list[:3], overwrite)